In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.utils import resample
import emot
from sklearn.svm import SVC
from sklearn.datasets import make_classification

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC



/home/lucia/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/lucia/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Importing the dataset
dataset = pd.read_csv('../data/status_dep_demog_big5_schwartz_swl_like.csv')
dataset[1:5]

,Unnamed: 0,Unnamed: 0.1,userid,text,Unnamed: 2,Unnamed: 3,date_added_x,time_completed_x,question_order_x,ethnicity,...,q50,q51,q52,q53,q54,q55,q56,q57,swl,like_id
1,1,1,05a1a1b67e3d7a923f9d0ee5502757c0,i cover my ears and scream at what i see quot ...,NaN,NaN,2011-04-11 22:41:00,2011-04-11 22:42:44,7\5\1\10\2\19\16\3\15\6\9\17\12\11\8\4\20\13\1...,White-American,...,9,2,6,9,6,7,5,7,2.8,"6a4355b020a885262933c999ae8e75de, 7304b068d93b..."
2,2,2,06165dddbf8856657d57d27600c4944c,long week in vacation bible school doing missi...,NaN,NaN,2011-06-22 11:36:47,2011-06-22 11:39:02,15\2\12\1\16\9\19\3\17\4\11\6\8\7\18\10\5\20\1...,White-American,...,9,9,9,9,9,8,7,9,4.0,"3cbcdc9163fc20e50d8d6f8a9a0d956b, 4484f3edaf8a..."
3,3,3,07244b3fd4536a8eea069ab02057e220,come tomorrow i will be home quot and therefo...,NaN,NaN,2011-04-26 08:21:57,2011-04-26 08:23:39,10\15\4\12\16\1\7\9\2\17\13\6\19\14\11\3\18\20...,White-other nationalities,...,9,1,9,9,9,5,3,9,5.8,"0fb5071a5dc785b06c7aad5b8f440ffe, 3990241cc719..."
4,4,4,0724fe854bd455061ba84efecdeff469,"it's time i fire up this life i'm livin' in!, ...",NaN,NaN,2011-02-21 00:26:08,2011-02-21 00:27:07,19\17\15\20\5\11\2\7\1\12\18\6\10\14\4\3\9\16\...,White-other nationalities,...,5,3,9,8,5,9,8,3,6.4,"3cbcdc9163fc20e50d8d6f8a9a0d956b, 558e3800b855..."


In [3]:
#this is the self-labeled file
# In[45]:
data = pd.read_csv('../data/self_label_distortion2.csv')
data[1:5]

,text,userid,negative_yn_self,distortion_yn,quote,magnitude,distortion.category,negative_yn.y,Positive,Negative
1,I messed up so bad and said things didn't mean...,1729ac1c5673a08e2d6b6326f5ad5537,1,1,NaN,4.0,labeling/all-or-nothing,yes,3,-2
2,< |I'm gonna live|I'm gonna survive|Don't want...,cb02460576e27f95654a9a0ef548fe5b,1,2,y,3.0,NaN,yes,4,-3
3,||if home's where my heart is then I'm out of ...,cb02460576e27f95654a9a0ef548fe5b,1,2,NaN,2.0,NaN,yes,1,-3
4,"Ah, the joy of seeing movie that nobody ACTUAL...",82e932005d8c94bb93bcb54fa19f421c,1,2,NaN,1.0,NaN,no,3,-1


In [4]:
####this function extract emoticons from the text and return a df with emoticons
def extract_emoticon(text):
    emo_fea = []
    for i in text:
        emo = emot.emoticons(i)
        if len(emo) > 0:
            emo_fea.append(emo[0]['value'])
        else:
            emo_fea.append(False)
            emo_df = pd.DataFrame(emo_fea)
    return emo_df

###process data

def preprocess(sent):

    words = str(sent).split()
    new_words = []
    # ps = PorterStemmer()
    
    for w in words:
        w = w.lower().replace("**bobsnewline**","")
        # remove non English word characters
        w = re.sub(r'[^\x00-\x7F]+',' ', w)
        # remove puncutation 
        w = re.sub(r'[^\w\s]','',w)
        # w = ps.stem(w)
        new_words.append(w)
        
    return ' '.join(new_words)

emo_df = extract_emoticon(data['text'])

        

In [5]:
#Add emoticon column to the feature df
frames = [data, emo_df]
data1 = pd.concat(frames, axis = 1)
data1.columns.values[10] = 'emoticon'


In [6]:
#remove na in id
data2 = data1[pd.notnull(data1['userid'])]
#senti strength score
data2['senti_score'] = data2['Positive'] + data2['Negative']

###merge text, id with all FB features
all_data = pd.merge(data2, dataset, on = 'userid', how = 'inner')
all_data[1:5]
all_data.shape

(912, 130)

In [9]:
#####convert categorical data 
#select useful variables in dataset
selected = ['userid', 'marital_status', 'ethnicity', 'gender','age','relationship_status', 'network_size','negative_yn_self', 'negative_yn','text_x','Positive','Negative','senti_score','distortion_yn', 'magnitude']
data_n = all_data.loc[:, (selected)]
data_dep = data_n
data_dep = data_dep[pd.notnull(data_dep['text_x'])]

#fill na with mean
data_dep['ethnicity'] = data_dep['ethnicity'].fillna('Other')
data_dep['marital_status'] = data_dep['marital_status'].fillna('Other')
data_dep['age'] = data_dep['age'].fillna(data_dep['age'].mean())
data_dep['relationship_status'] = data_dep['relationship_status'].fillna(0)
data_dep['network_size'] = data_dep['network_size'].fillna(data_dep['network_size'].mean())

####one hot encoding
features_oneHot = ['marital_status', 'ethnicity', 'gender','relationship_status']

x = data_dep[features_oneHot].values
y = data_dep['negative_yn_self'].values
# Encoding categorical data

marital_status = pd.get_dummies(x[:, 0])
ethnicity = pd.get_dummies(x[:, 1])
gender = pd.get_dummies(x[:, 2])
relationship_status = pd.get_dummies(x[:, 3])
#emoticon = pd.get_dummies(x[:, 4])

fea = pd.concat([marital_status, ethnicity, gender,relationship_status], axis =1).values
fea[1:5]

/home/lucia/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0]], dtype=uint8)

In [10]:
#add one hot feature with other features
features = ['age','network_size','Positive','Negative','senti_score'] 
x2 = data_dep[features].values

fb_fea = np.concatenate((fea, x2), axis=1)
##add LIWC
text_id = data_dep[['text_x','userid']]

###this is the text to be processed by LIWC  
data_dep['text_x'].to_csv('../data/liwc.csv')
# select LIWC data, scale it to remove 0
text_liwc = pd.read_csv('../data/distortion_liwc_1000.csv')
liwc = text_liwc.loc[:,'function':'OtherP'].values
#Center to the mean and component wise scale to unit variance.
#liwc_scaled = preprocessing.scale(liwc)
#liwc_scaled

#here we use word vector as features only

In [11]:
###word embedding text as feature
cv = CountVectorizer()
text_vec = cv.fit_transform(text_liwc['text']).toarray()

####convert to tfidf
tfidf_transformer = TfidfTransformer()
text_vec = tfidf_transformer.fit_transform(text_vec).toarray()

####combine with all features
X = text_vec

In [40]:
#Normalize data with standardscaler
scaler = StandardScaler()

In [53]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
##0.6995614035087719  best result
#{'svc__C': 1.5, 'svc__class_weight': 'balanced', 'svc__gamma': 0.0001, 'svc__kernel': 'sigmoid'}
cv = StratifiedKFold(n_splits=5)
svc = make_pipeline(StandardScaler(),svm.SVC())
parameters = [{'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'svc__gamma': [0.01, 0.001, 0.0001],
                     'svc__C':[0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.5, 2.0,] , 'svc__class_weight':['balanced']}]
                   
grid_search_item = GridSearchCV(estimator = svc,
                          param_grid = parameters,
                           cv =  cv,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.6995614035087719
{'svc__C': 1.5, 'svc__class_weight': 'balanced', 'svc__gamma': 0.0001, 'svc__kernel': 'sigmoid'}
0.694079 (0.023553) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'linear'}
0.664474 (0.005592) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'poly'}
0.336623 (0.001781) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}
0.683114 (0.038260) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'sigmoid'}
0.694079 (0.023553) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.001, 'svc__kernel': 'linear'}
0.666667 (0.004478) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.001, 'svc__kernel': 'poly'}
0.336623 (0.001781) with: {'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.337719 (0.002190) with: {'svc__C': 0.1, 'svc__cl

In [54]:
svc = make_pipeline(StandardScaler(),svm.SVC(gamma=0.001, class_weight='balanced', C = 1.5, kernel = 'sigmoid'))
scores = cross_val_score(svc, X, y, cv=cv, scoring='f1_weighted')
print(scores)
print("f1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
precision = cross_val_score(svc, X, y, cv=cv, scoring='precision')
recall = cross_val_score(svc, X, y, cv=cv, scoring='recall')
print(precision.mean())
print(rec8all.mean())

[0.66478799 0.59938767 0.68967462 0.68940601 0.67767971]
f1: 0.66 (+/- 0.07)
0.4952841882744218
0.6058170280274987


In [55]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
clf = make_pipeline(StandardScaler(),SGDClassifier(max_iter= 1000))
#clf = SGDClassifier(max_iter= 1000)
parameters = [{'sgdclassifier__alpha': [0.01, 0.05, 0.001, 0.005], 'sgdclassifier__class_weight':['balanced'],
              'sgdclassifier__loss': ['hinge','log','modified_huber','squared_hinge', 'perceptron'], 
               'sgdclassifier__penalty':['none','l1','l2']}]
                   
grid_search_item = GridSearchCV(clf,
                          param_grid = parameters,
                           scoring = 'accuracy',
                           cv = cv,
                           n_jobs = -1)
grid_search = grid_search_item.fit(X, y)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

0.6644736842105263
{'sgdclassifier__alpha': 0.005, 'sgdclassifier__class_weight': 'balanced', 'sgdclassifier__loss': 'modified_huber', 'sgdclassifier__penalty': 'l2'}


#text feature: wordvec + LIWC (no feature selection)

In [83]:
####combine with liwc
X_liwc = np.concatenate((X, liwc), axis=1)

In [84]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
##0.6995614035087719  best result
#{'svc__C': 1.5, 'svc__class_weight': 'balanced', 'svc__gamma': 0.0001, 'svc__kernel': 'sigmoid'}
cv = StratifiedKFold(n_splits=5, random_state = 0)
svc = make_pipeline(StandardScaler(),svm.SVC())
parameters = [{'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'svc__gamma': [0.01, 0.001, 0.0001],
                     'svc__C':[0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.5, 2.0,] , 'svc__class_weight':['balanced']}]
                   
grid_search_item = GridSearchCV(estimator = svc,
                          param_grid = parameters,
                           cv =  cv,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_liwc, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']

0.7302631578947368
{'svc__C': 2.0, 'svc__class_weight': 'balanced', 'svc__gamma': 0.0001, 'svc__kernel': 'sigmoid'}


In [86]:
svc = make_pipeline(StandardScaler(),svm.SVC(gamma=0.0001, class_weight='balanced', C = 2, kernel = 'sigmoid'))
scores = cross_val_score(svc, X_liwc, y, cv=cv, scoring='f1_weighted')
print(scores)
print("f1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
precision = cross_val_score(svc, X_liwc, y, cv=cv, scoring='precision')
recall = cross_val_score(svc, X_liwc, y, cv=cv, scoring='recall')
print(precision.mean())
print(recall.mean())  #recall too low

[0.74747119 0.66712111 0.74568893 0.68807222 0.74376738]
f1: 0.72 (+/- 0.07)
0.6400892074459856
0.4692226335272342


#text feature: wordvec + LIWC + demo 'marital_status', 'ethnicity', 'gender','relationship_status'

In [87]:
####combine with liwc
X_demog = np.concatenate((X_liwc, fea), axis=1)

In [88]:
X_demog.shape

(912, 5517)

In [89]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
svc = make_pipeline(StandardScaler(),svm.SVC())
parameters = [{'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'svc__gamma': [0.01, 0.001, 0.0001],
                     'svc__C':[0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.5, 2.0,] , 'svc__class_weight':['balanced']}]
                   
grid_search_item = GridSearchCV(estimator = svc,
                          param_grid = parameters,
                           cv =  cv,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_demog, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']

0.7247807017543859
{'svc__C': 1.5, 'svc__class_weight': 'balanced', 'svc__gamma': 0.0001, 'svc__kernel': 'sigmoid'}


In [90]:
svc = make_pipeline(StandardScaler(),svm.SVC(gamma=0.0001, class_weight='balanced', C = 1.5, kernel = 'sigmoid'))
scores = cross_val_score(svc, X_demog, y, cv=cv, scoring='f1_weighted')
print(scores)
print("f1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
precision = cross_val_score(svc, X_demog, y, cv=cv, scoring='precision')
recall = cross_val_score(svc, X_demog, y, cv=cv, scoring='recall')
print(precision.mean())
print(recall.mean())  #recall too low

[0.73884808 0.65345851 0.73294114 0.72003085 0.72047351]
f1: 0.71 (+/- 0.06)
0.6252197749295839
0.4660497091485986


Let's do some feature selection

In [92]:
svc = make_pipeline(StandardScaler(), SelectFromModel(LinearSVC(C=1.5, penalty="l2", dual=False)), svm.SVC(gamma=0.0001, class_weight='balanced', C = 1.5, kernel = 'sigmoid'))
parameters = [{'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'svc__gamma': [0.01, 0.001, 0.0001],
                     'svc__C':[0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.5, 2.0,] , 'svc__class_weight':['balanced']}]
                   
grid_search_item = GridSearchCV(estimator = svc,
                          param_grid = parameters,
                           cv =  cv,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_demog, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']

0.7214912280701754
{'svc__C': 0.1, 'svc__class_weight': 'balanced', 'svc__gamma': 0.01, 'svc__kernel': 'sigmoid'}


In [93]:
svc = make_pipeline(StandardScaler(), SelectFromModel(LinearSVC(C=1.5, penalty="l2", dual=False)), svm.SVC(gamma=0.01, class_weight='balanced', C = 0.1, kernel = 'sigmoid'))
scores = cross_val_score(svc, X_demog, y, cv=cv, scoring='f1_weighted')
print(scores)
print("f1: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
precision = cross_val_score(svc, X_demog, y, cv=cv, scoring='precision')
recall = cross_val_score(svc, X_demog, y, cv=cv, scoring='recall')
print(precision.mean())
print(recall.mean())

[0.73862186 0.6629387  0.74119275 0.7429567  0.74914068]
f1: 0.73 (+/- 0.06)
0.5716852063884027
0.7004230565838181


#Wordvec + LIWC fea select

In [ ]:
svc = make_pipeline(StandardScaler(), SelectFromModel(LinearSVC(C=1.5, penalty="l2", dual=False)), svm.SVC(gamma=0.0001, class_weight='balanced', C = 1.5, kernel = 'sigmoid'))
parameters = [{'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'svc__gamma': [0.01, 0.001, 0.0001],
                     'svc__C':[0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.5, 2.0,] , 'svc__class_weight':['balanced']}]
                   
grid_search_item = GridSearchCV(estimator = svc,
                          param_grid = parameters,
                           cv =  cv,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_demog, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']

In [94]:
X_liwc.shape

(912, 5455)

##fast text

In [1]:
import fastText


In [2]:
help(fastText.FastText)

Help on module fastText.FastText in fastText:

NAME
    fastText.FastText

DESCRIPTION
    # Copyright (c) 2017-present, Facebook, Inc.
    # All rights reserved.
    #
    # This source code is licensed under the BSD-style license found in the
    # LICENSE file in the root directory of this source tree. An additional grant
    # of patent rights can be found in the PATENTS file in the same directory.

FUNCTIONS
    load_model(path)
        Load a model given a filepath and return a model object.
    
    tokenize(text)
        Given a string of text, tokenize it and return a list of tokens
    
    train_supervised(input, lr=0.1, dim=100, ws=5, epoch=5, minCount=1, minCountLabel=0, minn=0, maxn=0, neg=5, wordNgrams=1, loss='softmax', bucket=2000000, thread=3, lrUpdateRate=100, t=0.0001, label='__label__', verbose=2, pretrainedVectors='')
        Train a supervised model and return a model object.
        
        input must be a filepath. The input text does not need to be tokenized
